In [1]:
import os
import numpy as np
import math
import pickle
import natsort
from probreg import cpd, gmmtree, filterreg, bcpd, math_utils
import copy
import pandas as pd
from scipy.spatial import KDTree
from scipy.stats import gaussian_kde
import tifffile as tf
import cv2

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import platform
if "windows" in platform.system().lower():
    data_root_dir = r"\\mfad\researchmn\HCPR\HCPR-GYNECOLOGICALTUMORMICROENVIRONMENT\Archive\WSIs\Ovarian_TMA"
elif "linux" in platform.system().lower():
    data_root_dir = "/infodev1/non-phi-data/junjiang/Ovarian_TMA"
else:
    raise Exception("platform not defined")

def get_TMA_core_list(img_path: str) -> list:
    img_fn_list = os.listdir(img_path)
    roi_list = [i.split("_")[0] for i in img_fn_list]
    return list(set(roi_list))

ROI_list = natsort.natsorted(get_TMA_core_list(os.path.join(data_root_dir, "AlignmentEval", "Sec1GroundTruth")))

In [3]:
def load_trans(trans_fn):
    fp = open(trans_fn, 'rb')
    data = pickle.load(fp)
    fp.close()
    return data

def get_cell_loc(HE_quant_fn):
    HE_quant_df = pd.read_csv(HE_quant_fn, sep='\t')
    he_x = HE_quant_df["Centroid X µm"]
    he_y = HE_quant_df["Centroid Y µm"]
    source = np.array([he_x, he_y]).T
    return source

def apply_aff_trans2points(source, M):
    '''
    source: cell centroids from H&E segmentation (N*2), N is the number of cell
    '''
    MM = np.vstack((M,np.array([0,0,1])))
    new_XY = np.vstack((np.transpose(source), np.ones((1, len(source)))))
    transformed_loc = np.dot(M, new_XY)
    affined_points = np.transpose(transformed_loc[0:2, :])
    return affined_points

# def apply_aff_trans2img(he_img, dapi_img, M):
#     aff_image = cv2.warpAffine(src=he_img, M=M, dsize=dapi_img.shape)
#     return aff_image

def calculate_points_affine_rmse(target_points, affined_points, leafsize=10):
    target_tree = KDTree(target_points, leafsize=leafsize)
    rmse = math_utils.compute_rmse(affined_points, target_tree)
    return rmse



In [6]:
HE_pxiel_size = 0.2201  # unit micron
MxIF_pixel_size = 0.325

pix_scale = HE_pxiel_size / MxIF_pixel_size
target_pixel_size = MxIF_pixel_size

sec_list = ["Sec1", "Sec2"]
seg_method_list = ["StarDist", "Watershed"]

for roi_id in ROI_list:
    MxIF_img_dir =os.path.join(data_root_dir, "FOVs", "MxIF_FOVs", "Slide2050_24Plex")
    mxif_fn = os.path.join(MxIF_img_dir, roi_id + ".tif")
    mxif_img = tf.TiffFile(mxif_fn)
    dapi_img = mxif_img.pages[0].asarray().astype(float)
    
    for sec in sec_list:
        if sec == "Sec1":
            HE_img_dir = os.path.join(data_root_dir, "FOVs", "HE_FOVs", "same_section")            
            # Get image
            he_fn = os.path.join(HE_img_dir, roi_id + ".tif")
            he_img = tf.TiffFile(he_fn).pages[0].asarray().astype(float)
            
            # Load ground truth transformation
            ground_truth_affine_dir = os.path.join(data_root_dir, "AlignmentEval", "Sec1GroundTruth")
            [gt_theta, gt_degrees, gt_s, gt_delta, gt_M, _] = load_trans(os.path.join(ground_truth_affine_dir, roi_id+"_trans.dat"))
            
            output_dir = os.path.join(data_root_dir, "AlignmentEval", "ApplyAlignment", "Sec1Output")
            
            for seg in seg_method_list:
                print("Running %s, %s for %s" % (roi_id, sec, seg))
                if seg == "StarDist":
                    HE_points_dir = os.path.join(data_root_dir, "AlignmentEval", "QuPathAnnoProj_HE_Sec1", "export")
                    MxIF_points_dir =os.path.join(data_root_dir, "AlignmentEval", "QuPathAnnoProj_MxIF", "export")
                    cpd_affine_dir = os.path.join(data_root_dir, "AlignmentEval", "Sec1_stardist_CPD")
                elif seg == "Watershed":
                    HE_points_dir = os.path.join(data_root_dir, "AlignmentEval", "QuPathAnnoProj_HE_Sec1_watershed", "export")
                    MxIF_points_dir =os.path.join(data_root_dir, "AlignmentEval", "QuPathAnnoProj_MxIF_watershed", "export")
                    cpd_affine_dir = os.path.join(data_root_dir, "AlignmentEval", "Sec1_watershed_CPD")
                else:
                    raise Exception("Unknown segmentation method")
                
                # Get points
                HE_quant_fn = os.path.join(HE_points_dir, roi_id + "_" + seg + "_QUANT.tsv")
                HE_points = get_cell_loc(HE_quant_fn)
                MxIF_quant_fn = os.path.join(MxIF_points_dir, roi_id + "_" + seg + "_QUANT.tsv")
                MxIF_points = get_cell_loc(MxIF_quant_fn)
                
                # apply transformation (from annotation) to points (from different cell segmentation methods) and image
                gt_affined_points = apply_aff_trans2points(HE_points, gt_M)
                gt_affined_HE_img = cv2.warpAffine(src=he_img, M=gt_M, dsize=dapi_img.shape)

                # apply transformation (from CPD) to points (from different cell segmentation methods) and image
                [theta, degrees, s, delta, M] = load_trans(os.path.join(cpd_affine_dir, roi_id+"_trans.dat"))
                cpd_affined_points = apply_aff_trans2points(HE_points, M)
                cpd_affined_HE_img = cv2.warpAffine(src=he_img, M=M, dsize=dapi_img.shape)
                
                rmse_gt = calculate_points_affine_rmse(MxIF_points, gt_affined_points, leafsize=10)
                rmse_cpd = calculate_points_affine_rmse(MxIF_points, cpd_affined_points, leafsize=10)
                
                # Save metrics 
                csv_fn = os.path.join(output_dir, roi_id + "_" + seg + "_metrics.csv")
                fp = open(csv_fn, "w")
                header_line = "gt_theta,gt_degrees,gt_s,gt_delta,rmse_gt,theta,degrees,s,delta,rmse_cpd\n"
                sv_list = [gt_theta,gt_degrees,gt_s,gt_delta,rmse_gt,theta,degrees,s,delta,rmse_cpd]
                data_line =  ",".join([str(i) for i in sv_list])
                fp.write(header_line + data_line[:-1] + "\n")
                # Save images
                res = (10000 / target_pixel_size, 10000 / target_pixel_size)  # convert um to cm for saving
                gt_affine_HE_fn = os.path.join(output_dir, roi_id + "_gt_affined.tif")
                if not os.path.exists(gt_affine_HE_fn): # TODO: Check if the resolution of the gt_affine_HE is correct!!!!
                    tf.imwrite(gt_affine_HE_fn, gt_affined_HE_img, photometric='rgb', resolution=res, resolutionunit="CENTIMETER")
                cpt_affine_HE_fn = os.path.join(output_dir, roi_id + "_" + seg + "_cpd_affined.tif")
                if not os.path.exists(cpt_affine_HE_fn):
                    tf.imwrite(cpt_affine_HE_fn, cpd_affined_HE_img, photometric='rgb', resolution=res, resolutionunit="CENTIMETER")
                
                # Save affined points
                affined_points_fn = os.path.join(output_dir, roi_id + "_"+ seg +"_affined_points.csv")
                fp = open(affined_points_fn, "w")
                wrt_str = "gt_aff_x,gt_aff_y,seg_aff_x,seg_aff_y\n"
                for p_idx, _ in enumerate(cpd_affined_points):
                    data_line_str = [str(gt_affined_points[p_idx, 0]), str(gt_affined_points[p_idx, 1]), \
                                     str(cpd_affined_points[p_idx, 0]), str(cpd_affined_points[p_idx, 1])]
                    data_line =  ",".join([str(i) for i in data_line_str])
                    wrt_str += data_line[:-1] + "\n"
                fp.write(wrt_str)
                
        elif sec == "Sec2":
            HE_img_dir = os.path.join(data_root_dir, "FOVs", "HE_FOVs", "Rab_Spine-22R919-A-SERBSVG-5X-08")

            # Get image
            he_fn = os.path.join(HE_img_dir, roi_id + ".tif")
            he_img = tf.TiffFile(he_fn).pages[0].asarray().astype(float)
            
            # Load ground truth transformation
            ground_truth_affine_dir = os.path.join(data_root_dir, "AlignmentEval", "Sec2GroundTruth")
            [gt_theta, gt_degrees, gt_s, gt_delta, gt_M, _] = load_trans(os.path.join(ground_truth_affine_dir, roi_id+"_trans.dat"))
            
            output_dir = os.path.join(data_root_dir, "AlignmentEval", "ApplyAlignment", "Sec2Output")
            
            for seg in seg_method_list:
                print("Running %s, %s for %s" % (roi_id, sec, seg))
                
                if seg == "StarDist":
                    HE_points_dir = os.path.join(data_root_dir, "AlignmentEval", "QuPathAnnoProj_HE_Sec2", "export")
                    MxIF_points_dir =os.path.join(data_root_dir, "AlignmentEval", "QuPathAnnoProj_MxIF", "export")
                    cpd_affine_dir = os.path.join(data_root_dir, "AlignmentEval", "Sec2_stardist_CPD")
                elif seg == "Watershed":
                    HE_points_dir = os.path.join(data_root_dir, "AlignmentEval", "QuPathAnnoProj_HE_Sec2_watershed", "export")
                    MxIF_points_dir =os.path.join(data_root_dir, "AlignmentEval", "QuPathAnnoProj_MxIF_watershed", "export")
                    cpd_affine_dir = os.path.join(data_root_dir, "AlignmentEval", "Sec2_watershed_CPD")
                else:
                    raise Exception("Unknown segmentation method")
                
                # Get points
                HE_quant_fn = os.path.join(HE_points_dir, roi_id + "_" + seg + "_QUANT.tsv")
                HE_points = get_cell_loc(HE_quant_fn)
                MxIF_quant_fn = os.path.join(MxIF_points_dir, roi_id + "_" + seg + "_QUANT.tsv")
                MxIF_points = get_cell_loc(MxIF_quant_fn)
                
                # apply transformation (from annotation) to points (from different cell segmentation methods) and image
                gt_affined_points = apply_aff_trans2points(HE_points, gt_M)
                gt_affined_HE_img = cv2.warpAffine(src=he_img, M=gt_M, dsize=dapi_img.shape)

                # apply transformation (from CPD) to points (from different cell segmentation methods) and image
                [theta, degrees, s, delta, M] = load_trans(os.path.join(cpd_affine_dir, roi_id+"_trans.dat"))
                cpd_affined_points = apply_aff_trans2points(HE_points, M)
                cpd_affined_HE_img = cv2.warpAffine(src=he_img, M=M, dsize=dapi_img.shape)
                
                rmse_gt = calculate_points_affine_rmse(MxIF_points, gt_affined_points, leafsize=10)
                rmse_cpd = calculate_points_affine_rmse(MxIF_points, cpd_affined_points, leafsize=10)
                
                # Save metrics 
                csv_fn = os.path.join(output_dir, roi_id + "_" + seg + "_metrics.csv")
                fp = open(csv_fn, "w")
                header_line = "gt_theta,gt_degrees,gt_s,gt_delta,rmse_gt,theta,degrees,s,delta,rmse_cpd\n"
                sv_list = [gt_theta,gt_degrees,gt_s,gt_delta,rmse_gt,theta,degrees,s,delta,rmse_cpd]
                data_line =  ",".join([str(i) for i in sv_list])
                fp.write(header_line + data_line[:-1] + "\n")
                # Save images
                res = (10000 / target_pixel_size, 10000 / target_pixel_size)  # convert um to cm for saving
                gt_affine_HE_fn = os.path.join(output_dir, roi_id + "_gt_affined.tif")
                if not os.path.exists(gt_affine_HE_fn): # TODO: Check if the resolution of the gt_affine_HE is correct!!!!
                    tf.imwrite(gt_affine_HE_fn, gt_affined_HE_img, photometric='rgb', resolution=res, resolutionunit="CENTIMETER")
                cpt_affine_HE_fn = os.path.join(output_dir, roi_id + "_" + seg + "_cpd_affined.tif")
                if not os.path.exists(cpt_affine_HE_fn):
                    tf.imwrite(cpt_affine_HE_fn, cpd_affined_HE_img, photometric='rgb', resolution=res, resolutionunit="CENTIMETER")
                
                # Save affined points
                affined_points_fn = os.path.join(output_dir, roi_id + "_"+ seg +"_affined_points.csv")
                fp = open(affined_points_fn, "w")
                wrt_str = "gt_aff_x,gt_aff_y,seg_aff_x,seg_aff_y\n"
                for p_idx, _ in enumerate(cpd_affined_points):
                    data_line_str = [str(gt_affined_points[p_idx, 0]), str(gt_affined_points[p_idx, 1]), \
                                     str(cpd_affined_points[p_idx, 0]), str(cpd_affined_points[p_idx, 1])]
                    data_line =  ",".join([str(i) for i in data_line_str])
                    wrt_str += data_line[:-1] + "\n"
                fp.write(wrt_str)
        else:
            raise Exception("Unknown section name")


Running A-8, Sec1 for StarDist
Running A-8, Sec1 for Watershed
Running A-8, Sec2 for StarDist
Running A-8, Sec2 for Watershed
Running A-22, Sec1 for StarDist
Running A-22, Sec1 for Watershed
Running A-22, Sec2 for StarDist
Running A-22, Sec2 for Watershed
Running B-9, Sec1 for StarDist
Running B-9, Sec1 for Watershed
Running B-9, Sec2 for StarDist
Running B-9, Sec2 for Watershed
Running B-11, Sec1 for StarDist
Running B-11, Sec1 for Watershed
Running B-11, Sec2 for StarDist
Running B-11, Sec2 for Watershed
Running B-12, Sec1 for StarDist
Running B-12, Sec1 for Watershed
Running B-12, Sec2 for StarDist
Running B-12, Sec2 for Watershed
Running B-15, Sec1 for StarDist
Running B-15, Sec1 for Watershed
Running B-15, Sec2 for StarDist
Running B-15, Sec2 for Watershed
Running B-21, Sec1 for StarDist
Running B-21, Sec1 for Watershed
Running B-21, Sec2 for StarDist
Running B-21, Sec2 for Watershed
Running C-10, Sec1 for StarDist
Running C-10, Sec1 for Watershed
Running C-10, Sec2 for StarDist
R